In [2]:
import pandas as pd
# Load the file to inspect its structure
df = pd.read_csv('2-3Apr X scrape.csv', encoding='ISO-8859-1')
df.head()

,ï»¿createdAt,fullText,author_id,author_location
0,Thu Apr 03 23:59:57 +0000 2025,@SpencerHakimian Tariffs arenât self-taxatio...,1487236076548894724,"Issaquah, WA"
1,Thu Apr 03 23:59:56 +0000 2025,@joma_gc Veitnam has the audacity to have a 90...,1762610631348510720,NaN
2,Thu Apr 03 23:59:47 +0000 2025,My thoughts on the tariff debate:\n\n1) If the...,1126395138,California
3,Thu Apr 03 23:59:44 +0000 2025,@thejefflutz Grok has the total labor hours to...,4764020911,"Kenmore, WA"
4,Thu Apr 03 23:59:44 +0000 2025,"'India very, very tough. The Prime Minister ju...",1790517608153772033,Los Angeles California


In [4]:
import re

def clean_text_preprocessing1(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)     # Remove URLs
    text = re.sub(r"@\w+|#\w+", "", text)                   # Remove mentions and hashtags
    text = re.sub(r"[^\x00-\x7F]+", "", text)               # Remove emojis (non-ASCII characters)
    text = re.sub(r"\d+", "", text)                         # Remove numerical characters
    return text

# Apply this to DataFrame 
df['text_preprocessing1'] = df['fullText'].apply(clean_text_preprocessing1)
df[['fullText', 'text_preprocessing1']].head()

,fullText,text_preprocessing1
0,@SpencerHakimian Tariffs arenât self-taxatio...,"Tariffs arent self-taxation, theyre leverage...."
1,@joma_gc Veitnam has the audacity to have a 90...,Veitnam has the audacity to have a % tariff o...
2,My thoughts on the tariff debate:\n\n1) If the...,My thoughts on the tariff debate:\n\n) If they...
3,@thejefflutz Grok has the total labor hours to...,Grok has the total labor hours to build a car...
4,"'India very, very tough. The Prime Minister ju...","'India very, very tough. The Prime Minister ju..."


In [6]:
import string

def clean_text_preprocessing2(text):
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
    return text

# Apply this to the result of step 1
df['text_preprocessing2'] = df['text_preprocessing1'].apply(clean_text_preprocessing2)
df[['fullText', 'text_preprocessing1','text_preprocessing2']].head()

,fullText,text_preprocessing1,text_preprocessing2
0,@SpencerHakimian Tariffs arenât self-taxatio...,"Tariffs arent self-taxation, theyre leverage....",Tariffs arent selftaxation theyre leverage If...
1,@joma_gc Veitnam has the audacity to have a 90...,Veitnam has the audacity to have a % tariff o...,Veitnam has the audacity to have a tariff on...
2,My thoughts on the tariff debate:\n\n1) If the...,My thoughts on the tariff debate:\n\n) If they...,My thoughts on the tariff debate\n\n If they t...
3,@thejefflutz Grok has the total labor hours to...,Grok has the total labor hours to build a car...,Grok has the total labor hours to build a car...
4,"'India very, very tough. The Prime Minister ju...","'India very, very tough. The Prime Minister ju...",India very very tough The Prime Minister just ...


In [8]:
# Convert texts to lowercase
df['text_preprocessing2'] = \
df['text_preprocessing2'].map(lambda x: x.lower())

df['text_preprocessing2'].head()

0     tariffs arent selftaxation theyre leverage if...
1     veitnam has the audacity to have a  tariff on...
2    my thoughts on the tariff debate\n\n if they t...
3     grok has the total labor hours to build a car...
4    india very very tough the prime minister just ...
Name: text_preprocessing2, dtype: object

In [10]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in ENGLISH_STOP_WORDS]
    return " ".join(filtered)

# Apply to cleaned column
df['text_no_stopwords'] = df['text_preprocessing2'].apply(remove_stopwords)
df[['text_preprocessing2', 'text_no_stopwords']].head()

,text_preprocessing2,text_no_stopwords
0,tariffs arent selftaxation theyre leverage if...,tariffs arent selftaxation theyre leverage vie...
1,veitnam has the audacity to have a tariff on...,veitnam audacity tariff goods f
2,my thoughts on the tariff debate\n\n if they t...,thoughts tariff debate tariff law equality dic...
3,grok has the total labor hours to build a car...,grok total labor hours build car parts hr that...
4,india very very tough the prime minister just ...,india tough prime minister just left great fri...


In [12]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

# Download required resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
        
def lemmatize_with_pos(text):
    tokens = nltk.word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos)) 
        for word, pos in tagged_tokens
    ]
    return " ".join(lemmatized)
    
df['text_lemmatized'] = df['text_no_stopwords'].apply(lemmatize_with_pos)
df[['text_no_stopwords', 'text_lemmatized']].head()

[nltk_data] Downloading package punkt to /Users/patnbe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/patnbe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/patnbe/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/patnbe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text_no_stopwords,text_lemmatized
0,tariffs arent selftaxation theyre leverage vie...,tariff arent selftaxation theyre leverage viet...
1,veitnam audacity tariff goods f,veitnam audacity tariff good f
2,thoughts tariff debate tariff law equality dic...,thought tariff debate tariff law equality dict...
3,grok total labor hours build car parts hr that...,grok total labor hour build car part hr thats ...
4,india tough prime minister just left great fri...,india tough prime minister just leave great fr...


In [14]:
# Tokenize text
tokenized_text = df['text_lemmatized'].apply(lambda x: x.split())

In [16]:
from gensim import corpora

# Create Dictionary
id2word = corpora.Dictionary(tokenized_text)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tokenized_text]

In [18]:
from gensim.models.ldamodel import LdaModel

# Train LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10, # number of topics
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

In [20]:
# Display topics
topics = lda_model.print_topics()
for idx, topic in topics:
    print(f"Topic {idx + 1}: {topic}")

Topic 1: 0.095*"tariff" + 0.038*"good" + 0.019*"amp" + 0.017*"do" + 0.017*"make" + 0.016*"think" + 0.015*"american" + 0.014*"get" + 0.014*"country" + 0.014*"price"
Topic 2: 0.018*"sir" + 0.016*"maga" + 0.012*"whats" + 0.011*"tracker" + 0.010*"guy" + 0.010*"wait" + 0.009*"stop" + 0.008*"wont" + 0.008*"view" + 0.008*"ireland"
Topic 3: 0.021*"just" + 0.020*"trade" + 0.019*"update" + 0.019*"deficit" + 0.018*"break" + 0.016*"thing" + 0.015*"tariff" + 0.014*"south" + 0.014*"charge" + 0.011*"follow"
Topic 4: 0.022*"america" + 0.019*"really" + 0.017*"pay" + 0.016*"point" + 0.014*"dont" + 0.013*"nearly" + 0.010*"low" + 0.010*"post" + 0.010*"know" + 0.010*"far"
Topic 5: 0.028*"list" + 0.021*"heard" + 0.018*"mcdonald" + 0.016*"population" + 0.016*"island" + 0.013*"islands" + 0.012*"total" + 0.012*"just" + 0.011*"russia" + 0.010*"future"
Topic 6: 0.117*"tariff" + 0.078*"trump" + 0.024*"trade" + 0.022*"market" + 0.018*"import" + 0.017*"reciprocal" + 0.016*"president" + 0.015*"india" + 0.014*"donald

In [22]:
lda_model.get_document_topics(corpus)
for i, doc in enumerate(corpus):
    topics = lda_model.get_document_topics(doc)
    print(f"Document {i + 1} Topics: {topics}")

Document 1 Topics: [(0, 0.01584109), (5, 0.19528563), (8, 0.76663)]
Document 2 Topics: [(0, 0.28230765), (2, 0.023193317), (3, 0.016584728), (4, 0.012166404), (5, 0.18184888), (6, 0.02028767), (8, 0.44460252)]
Document 3 Topics: [(0, 0.1423565), (5, 0.24484311), (6, 0.5813017)]
Document 4 Topics: [(0, 0.16222039), (4, 0.76796675), (5, 0.043339346)]
Document 5 Topics: [(0, 0.020709816), (2, 0.69766754), (5, 0.25260216)]
Document 6 Topics: [(0, 0.368613), (2, 0.054140646), (3, 0.40398818), (5, 0.068647206), (8, 0.089470774)]
Document 7 Topics: [(0, 0.03805203), (2, 0.013588205), (5, 0.24748407), (6, 0.6621029), (8, 0.010773865)]
Document 8 Topics: [(0, 0.03664072), (2, 0.013644677), (5, 0.21129264), (6, 0.6996672), (8, 0.010762649)]
Document 9 Topics: [(0, 0.02023733), (5, 0.946136)]
Document 10 Topics: [(0, 0.07620554), (3, 0.72656906), (5, 0.16738296)]
Document 11 Topics: [(0, 0.92267185), (5, 0.04370524)]
Document 12 Topics: [(0, 0.032312695), (2, 0.012548266), (5, 0.9083641), (6, 0.0

In [24]:
from gensim.models.coherencemodel import CoherenceModel

# Compute c_v coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_text, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print(f"C_v coherence score: {coherence_lda:.4f}")

C_v coherence score: 0.4505


In [26]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Prepare the visualization
lda_vis = gensimvis.prepare(lda_model, corpus, id2word)

pyLDAvis.display(lda_vis)

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=58409) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=58409) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=58409) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=58409) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarnin